### This file contains the default proposed method with 9 new clustering seeds.

1. The best number of clusters will be printed out during execution.

2. The results will be stored in `/Reproduction/Results/CRS/`. Inside this folder, the second-layer folders will be named by the random seeds for data splitting. Inside each of these folders, the third-layer folders contains the raw results of the proposed method with 9 new clustering seeds. For example, `/Reproduction/Results/CRS/0/0-1-ResultData/` is the folder for the results of the proposed method with splitting seed 0 and clustering seed 1.

In [ ]:
import os
import sklearn
import sklearn.cluster
import pandas as pd
import numpy as np
import datetime
from kneed import KneeLocator

pd.options.mode.chained_assignment = None

In [ ]:
# Get the current directory.
current_dir = os.getcwd()

# Basic fuction for generating query and template series.
def split_sequence_with_normalisation(sequence, n_steps_infunction):
    X, y, sequence_mean, sequence_std = list(), list(), list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_infunction
        if end_ix > len(sequence)-1:
            break
        seq_x_original, seq_y = sequence[i:end_ix], sequence[end_ix]
        seq_x_mean, seq_x_std = np.mean(seq_x_original), np.std(seq_x_original)
        seq_x = (seq_x_original - seq_x_mean) / seq_x_std
        X.append(seq_x)
        y.append(seq_y)
        sequence_mean.append(seq_x_mean)
        sequence_std.append(seq_x_std)
    return np.array(X), np.array(y), np.array(sequence_mean), np.array(sequence_std)

# Set up random seeds for data splitting and clustering.
split_rs = [290, 150, 266, 78, 148, 133, 155, 135, 178, 241]
new_cluster_rs = [1, 186, 333, 70, 63, 298, 159, 3, 62]

# Set up directories to store result data.
for rs in split_rs:
    for this_crs in new_cluster_rs:    
        os.makedirs(current_dir+'/Results/CRS/'+str(rs)+'/'+str(rs)+'-'+str(this_crs)+'-ResultData/')

# Set up data source and hyperparameters.
path = current_dir+'/Data14/'
template_length = 14
query_length = 14
this_m = 200
this_w3 = 1.1

In [ ]:
for rs in split_rs:
    print('Calculating seed', rs, 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    for this_crs in new_cluster_rs:
        ## Splitting data
        print('Calculating clustering seed', this_crs, 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            
        files = sorted(os.listdir(path))
        pads = [elt[:-7] for elt in files]
        files_df = pd.DataFrame({'filename':files, 'pad':pads})

        num_wells_in_pad_df = pd.DataFrame(files_df['pad'].value_counts()).reset_index()
        num_wells_in_pad_df.columns = ['pad', 'count']
        unique_pads = np.unique(pads)
        unique_pads_df = pd.DataFrame({'pad':unique_pads})
        unique_pads_df = pd.merge(unique_pads_df, num_wells_in_pad_df, on='pad')

        np.random.seed(rs)
        unique_pads_df_shuffled = unique_pads_df.sample(frac=1).reset_index(drop=True)
        counter = 0
        for idx in range(len(unique_pads_df_shuffled)):
            counter += unique_pads_df_shuffled['count'][idx]
            if counter >= 300:
                break
            else:
                continue
        end_of_training = idx

        train_files_shuffled = []
        for idx in range(end_of_training+1):
            pad_name = unique_pads_df_shuffled['pad'][idx]
            for file in files:
                if file[:-7] == pad_name:
                    train_files_shuffled.append(file)
                else:
                    continue

        test_files_shuffled = []
        for idx in range(end_of_training+1,len(unique_pads_df_shuffled)):
            pad_name = unique_pads_df_shuffled['pad'][idx]
            for file in files:
                if file[:-7] == pad_name:
                    test_files_shuffled.append(file)
                else:
                    continue

        print('len(train_files_shuffled):', len(train_files_shuffled), '          len(test_files_shuffled):', len(test_files_shuffled))
        print(test_files_shuffled)

        ## Calculating template set
        print('Calculating template_df', 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        template, forecast, template_mean, template_std = np.empty([1, template_length]), np.empty([1,]), np.empty([1,]), np.empty([1,])

        for i in range(len(train_files_shuffled)):
            temp_df = pd.read_excel(path+train_files_shuffled[i], header = 0, sheet_name = 0)
            reopenings = list(temp_df[temp_df['Mark'] == 'reopening'].index)
            reopenings = np.insert(reopenings, len(reopenings), len(temp_df))
            temp_df = temp_df['Q']/temp_df['t']
            for j in range(len(reopenings)-1):
                temp_sub_df = temp_df.iloc[reopenings[j]:reopenings[j+1]]
                temp_template, temp_forecast, temp_template_mean, temp_template_std = split_sequence_with_normalisation(temp_sub_df.values, template_length)
                template = np.concatenate((template, temp_template), axis=0)
                forecast = np.concatenate((forecast, temp_forecast), axis=0)
                template_mean = np.concatenate((template_mean, temp_template_mean), axis=0)
                template_std = np.concatenate((template_std, temp_template_std), axis=0)

        template, forecast, template_mean, template_std = template[1:], forecast[1:], template_mean[1:], template_std[1:]
        template_df = pd.DataFrame(template)
        forecast_df = pd.DataFrame(forecast)
        template_mean_df = pd.DataFrame(template_mean)
        template_std_df = pd.DataFrame(template_std)
        print(template_df.shape, forecast_df.shape, template_mean_df.shape, template_std_df.shape)

        ## Clustering
        print('Searching n_clusters', 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        scores = []
        n_clusters_to_search = np.linspace(50, 600, 12, dtype=int)
        for n_of_clusters in n_clusters_to_search:
            print('n_clusters=', n_of_clusters, 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            kmeans = sklearn.cluster.KMeans(n_clusters=n_of_clusters, max_iter=10000, tol=1e-6, random_state=this_crs).fit(template_df.values)
            scores.append(kmeans.inertia_)
        kn = KneeLocator(n_clusters_to_search, scores, curve='convex', direction='decreasing')
        best_n_clusters = kn.knee
        print('best_n_clusters:', best_n_clusters)

        print('Clustering', 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        kmeans = sklearn.cluster.KMeans(n_clusters=best_n_clusters, max_iter=10000, tol=1e-6, random_state=this_crs).fit(template_df.values)
        labels = kmeans.labels_
        clustering_result = pd.DataFrame()
        clustering_result['Template'] = template_df.index
        clustering_result['Cluster'] = labels
        for i in range(len(clustering_result)):
            clustering_result['Cluster'][i] = 'Cluster '+str(clustering_result['Cluster'][i]+1)

        cluster_centers_df = pd.DataFrame(kmeans.cluster_centers_)

        ## Forecasting
        print('Forecasting', 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        # Get a test well.
        for m in range(len(test_files_shuffled)):
        
            df = pd.read_excel(path+test_files_shuffled[m], header = 0, sheet_name = 0)
            df['q'] = df['Q']/df['t']

            print('=====Calculating well=====', m, test_files_shuffled[m], 'at', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            reopenings = list(df[df['Mark'] == 'reopening'].index)
            reopenings = np.insert(reopenings, len(reopenings), len(df))

            forecasts_multisteps_this_well = []
            y_true_all_this_well = []
            prod_times_this_well = []
            markers_this_well = []

            # Get a production stage.
            for l in range(len(reopenings)-1):
                sub_df = df.iloc[reopenings[l]:reopenings[l+1]]
                
                forecasts_multisteps = list(sub_df['q'][:template_length].values)
                prod_times = sub_df['t'].values
                markers = ['initial'] * template_length

                sub_df = sub_df['q']
                global_upper_bound = sub_df.iloc[:template_length].max() * this_w3
                query_all, y_true_all, query_mean_all, query_std_all = split_sequence_with_normalisation(sub_df.values, query_length)
                y_true_all_expanded = list(sub_df[:template_length].values) + list(y_true_all)
                
                query = query_all[0]
                query_mean = query_mean_all[0]
                query_std = query_std_all[0]
                for j in range(len(query_all)):
                    markers.append('forecast')

                    distances_to_cluster = []
                    for i in range(len(cluster_centers_df)):
                        template = cluster_centers_df.iloc[i,:].values
                        distances_to_cluster.append(np.linalg.norm(query-template))
                    neighbour = sorted(range(len(distances_to_cluster)), key=lambda idx: distances_to_cluster[idx])[0]
                    cluster_name_of_neighbour = 'Cluster '+str(neighbour+1)
                    idx_of_templates_to_search = clustering_result[clustering_result['Cluster'] == cluster_name_of_neighbour]['Template'].values

                    templates_to_search = template_df.iloc[idx_of_templates_to_search].reset_index(drop=True)
                    templates_to_search_mean = template_mean_df.iloc[idx_of_templates_to_search].reset_index(drop=True)
                    templates_to_search_std = template_std_df.iloc[idx_of_templates_to_search].reset_index(drop=True)
                    templates_to_search_forecast = forecast_df.iloc[idx_of_templates_to_search].reset_index(drop=True)

                    distances = []
                    for i in range(len(templates_to_search)):
                        template = templates_to_search.iloc[i,:].values
                        distances.append(np.linalg.norm(query-template))

                    neighbours = sorted(range(len(distances)), key=lambda idx: distances[idx])[:this_m]
                    variations = []
                    for i in range(len(neighbours)):
                        template_normalised = templates_to_search.iloc[neighbours[i],:]
                        this_template_mean = templates_to_search_mean.iloc[neighbours[i]].values
                        this_template_std = templates_to_search_std.iloc[neighbours[i]].values
                        this_forecast = templates_to_search_forecast.iloc[neighbours[i]].values
                        template_original = template_normalised*this_template_std + this_template_mean
                        
                        template_weights = []
                        template_length_sum = sum(range(1, template_length+1))
                        for k in range(1, len(template_original)+1):
                            template_weights.append(k/template_length_sum)
                        template_weighted_sum =  np.sum([a*b for a,b in zip(template_weights,template_original)])
                            
                        variation = (this_forecast-template_weighted_sum)/template_weighted_sum
                        variations.append(variation[0])

                    distance_weights = []
                    neighbour_distances = [distances[idx] for idx in neighbours]
                    neighbour_distances_sum = np.sum(neighbour_distances)
                    actual_num_neighbours = len(neighbours)
                    for i in range(len(neighbour_distances)):
                        distance_weights.append(neighbour_distances[actual_num_neighbours-1-i]/neighbour_distances_sum)
                    variation_forecast = np.sum([a*b for a,b in zip(distance_weights,variations)])
                    
                    query_original = query*query_std + query_mean
                    query_weights = []
                    query_length_sum = sum(range(1, query_length+1))
                    for k in range(1, len(query_original)+1):
                        query_weights.append(k/query_length_sum)
                    query_weighted_sum =  np.sum([a*b for a,b in zip(query_weights,query_original)])
                    forecast = query_weighted_sum*variation_forecast + query_weighted_sum
                    if forecast > global_upper_bound:
                        forecast = query_weighted_sum
                    forecasts_multisteps.append(forecast)
                    
                    query_original_new = np.append(query_original, forecast)
                    query_original_new = np.delete(query_original_new, 0)
                    query_mean, query_std = np.mean(query_original_new), np.std(query_original_new), 
                    if query_std == 0:
                        break
                    else:
                        query = (query_original_new - query_mean)/query_std
                
                for j2 in range(j+1, len(query_all)):
                    markers.append('forecast')
                    forecasts_multisteps.append(query_mean)

                for j in range(len(markers)):
                    y_true_all_this_well.append(y_true_all_expanded[j])
                    forecasts_multisteps_this_well.append(forecasts_multisteps[j])
                    prod_times_this_well.append(prod_times[j])
                    markers_this_well.append(markers[j])

            # Result
            multi_step_result_df = pd.DataFrame()
            multi_step_result_df['True'] = y_true_all_this_well
            multi_step_result_df['Pred'] = forecasts_multisteps_this_well
            multi_step_result_df['t'] = prod_times_this_well
            multi_step_result_df['Mark'] = markers_this_well
            multi_step_result_df['TrueCumu'] = (multi_step_result_df['True']*multi_step_result_df['t']).cumsum()
            multi_step_result_df['PredCumu'] = (multi_step_result_df['Pred']*multi_step_result_df['t']).cumsum()

            writer = pd.ExcelWriter(current_dir+'/Results/CRS/'+str(rs)+'/'+str(rs)+'-'+str(this_crs)+'-ResultData/ResultData-'+str(m)+'-'+str(test_files_shuffled[m]))
            multi_step_result_df.to_excel(writer, float_format='%.5f', header=True, index=False)
            writer.save()
            writer.close()